# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [20]:
#include <stdio.h>

int calcularParcelas(double S, int N, double J) {
    double Pmes = S;
    printf("Parcelas do financiamento:");
    for (int i = 0; i < N; i++) {
        printf(" %.2lf", Pmes);
        Pmes = Pmes * (1 + 0.01 * J);
    }
}

int main() {
    calcularParcelas(200.00, 5, 1.00);
    return 0;
}    

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [21]:
#include <stdio.h>

double proximaParcela(double S, int X, double J) {
    for (int i = 0; i < X; i++)
        S = S * (1 + 0.01 * J);
    return S;
}

int main() {
    int N = 5;
    printf("Parcelas do financiamento:");
    for (int i = 0; i < N; i++) {
        printf(" %.2lf", proximaParcela(200.00, i, 1.00));
    }
    return 0;
}

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [22]:
#include <stdio.h>

typedef struct {
    double S;
    int N;
    double J;
    int i;
} emprestimo;

emprestimo iniciarEmprestimo(double S, int N, double J) {
    emprestimo emp;
    emp.S = S;
    emp.N = N;
    emp.J = J;
    emp.i = 0;
    return emp;
}    

double proximaParcela(emprestimo * p_emp) {
    double parcela = p_emp->S;
    for (int j = 0; j < p_emp->i; j++)
        parcela = parcela * ((1 + 0.01 * p_emp->J));
    p_emp->i++;
    return parcela;
}

int main() {
    emprestimo emp = iniciarEmprestimo(200.00, 5, 1);
    printf("Parcelas do financiamento:");
    for (int i = 0; i < emp.N; i++)
        printf(" %.2lf", proximaParcela(&emp));
    return 0;
}


Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [23]:
#include <stdio.h>
#include <stdlib.h>

typedef struct {
    int N;
    double J;
    int atual;
    double * parcelas;
} emprestimo;

emprestimo iniciarEmprestimo(double S, int N, double J) {
    emprestimo emp;
    emp.N = N;
    emp.J = J;
    emp.atual = 0;
    emp.parcelas = malloc(N * sizeof(double));
    emp.parcelas[0] = S;
    return emp;
}    

double proximaParcela(emprestimo * p_emp) {
    p_emp->atual++;
    double ant = p_emp->parcelas[p_emp->atual - 1];
    double S = p_emp->parcelas[0];

    if (p_emp->atual != 1)
        p_emp->parcelas[p_emp->atual] = ant * ((1 + 0.01 * p_emp->J));
    else
        p_emp->parcelas[1] = S;
    return p_emp->parcelas[p_emp->atual];
}

int main() {
    emprestimo emp = iniciarEmprestimo(200.00, 5, 1);
    printf("Parcelas do financiamento:");
    for (int i = 0; i < emp.N; i++)
        printf(" %.2lf", proximaParcela(&emp));
    free(emp.parcelas);
    return 0;
}


Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [24]:
#include <stdio.h>
#include <stdlib.h>

typedef struct {
    int N;
    double J;
    int atual;
    double * parcelas;
} emprestimo;

typedef struct {
    emprestimo * emps;
} controle;

emprestimo iniciarEmprestimo(double S, int N, double J) {
    emprestimo emp;
    emp.N = N;
    emp.J = J;
    emp.atual = 0;
    emp.parcelas = malloc(N * sizeof(double));
    emp.parcelas[0] = S;
    return emp;
}    

controle iniciarControle(int N) {
    controle ctrl;
    ctrl.emps = malloc(N * sizeof(emprestimo));
    return ctrl;
}

double proximaParcela(controle * p_ctrl, int j) {
    double parcela = 0;
    double S = 0;
    int atual;
    S = p_ctrl->emps[j].parcelas[0];
    p_ctrl->emps[j].atual++;
    atual = p_ctrl->emps[j].atual;
    if (atual != 1)
        p_ctrl->emps[j].parcelas[atual] = p_ctrl->emps[j].parcelas[atual - 1] * ((1 + 0.01 * p_ctrl->emps[j].J));
    else if (atual = 1)
        p_ctrl->emps[j].parcelas[atual] = S;
    return p_ctrl->emps[j].parcelas[atual];
}

int min(int a, int b) {
    return (a < b) ? a : b;
}

int max(int a, int b) {
    return (a > b) ? a : b;
}

void liberarMemoria(controle * p_ctrl) {
    free(p_ctrl->emps[0].parcelas);
    free(p_ctrl->emps[1].parcelas);
    free(p_ctrl->emps); 
}

int main() {
    controle ctrl = iniciarControle(2);
    ctrl.emps[0] = iniciarEmprestimo(200.00, 5, 1);
    ctrl.emps[1] = iniciarEmprestimo(500.00, 7, 2);
    int M = max(ctrl.emps[0].N, ctrl.emps[1].N);
    int N = min(ctrl.emps[0].N, ctrl.emps[1].N);
    int aux;
    for (int i = 1; i < N + 1; i++)
        for (aux = 1; aux < 3; aux++)
            printf("Emprestimo %d: parcela %d eh %.2lf\n", aux, i, proximaParcela(&ctrl, aux - 1));
    aux = (M == ctrl.emps[0].N) ? 1 : 2;
    for (int i = N + 1; i < M + 1; i++)
        printf("Emprestimo %d: parcela %d eh %.2lf\n", aux, i, proximaParcela(&ctrl, aux - 1));
    liberarMemoria(&ctrl);
    return 0;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08


munmap_chunk(): invalid pointer
[C kernel] Executable exited with code -6